<h1>Connect to your MongoDB database</h1>
<p>In principle, any number of users can access any number of MongoDB databases hosted on a single server. Some can have read permission, some can read and write, etc.</p>
<p>For this course, we have given each of you read/write access to a database whose name matches your NetID. Your default password is <tt>12345</tt> (we'll change this in the next step). In particular, this is separate from your Yale password. For simplicity, we'll store the netid and password as variables.</p>
<p><b>Be sure to change your netid</b> (and, later, your password) <b>before running this cell</b>:</p>

In [1]:
netid = 'testuser'
password = '12345'

<p>Now let's actually connect to the database. <tt>username</tt> is, of course, your user name and <tt>authSource</tt> is the name of the database; for this course they are both your NetID, but normally they will be different.</p>

In [2]:
from pymongo import MongoClient
mongodb = MongoClient(username=netid, password=password, authSource=netid)
mydb = mongodb[netid]

<p>If you wanted to connect to a database located elsewhere, you'd pass in the URL to the MongoClient constructor; for this class, however, we'll always use the one on the class server, so there is no need to specify.</p>

<h1>Change password</h1>
<p>Before we do anything else, let's change your MongoDB password. <b>Modify the following line with your new password.</b> Good security practice is to use a password you do not use anywhere else. (Note that the instructors have access to your Jupyter notebooks.)</p>

In [3]:
mydb.command('updateUser', netid, pwd='12345')

{'ok': 1.0}

<p>If this succeeded, you should see <tt>{'ok': 1.0}</tt>; otherwise you should get a <tt>pymongo.errors.OperationFailure</tt> exception.</p>

<h1>Define a collection</h1>
<p>A collection can be defined by simply referring to the <tt>database.collection_name</tt>. Here, we'll create one, assign it to a variable (for convenience), and print it out (just to see what happens). <i>Note: databases can be created similarly.</i></p> 

In [4]:
cells = mydb.cells # or, equivalently, mydb['cells']
print(cells)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='testuser'), 'testuser'), 'cells')


<p>Since MongoDB is by default schemaless, we're done.</p>
<p><i>Technically</i>, the collection hasn't been actually created yet since we haven't put anything in it. Thus if we ask for the list of collection names, we won't get anything yet.</p>

In [5]:
mydb.collection_names()

[]

<h1>Adding data to a collection</h1>

Use <tt>insert_one</tt> or <tt>insert_many</tt> to insert one or many documents into a collection:

In [6]:
cells.insert_one({
                  'name': 'Hippocampus CA1 pyramidal cell',
                  'channels': ['I Na,t', 'I Na,p', 'I Potassium', 'I A', 'I K', 'I M',
                               'I L high threshold', 'I N', 'I T low threshold', 'I p,q',
                               'I K,Ca', 'I h'],
                  'transmitters': ['NO', 'Glutamate'],
                  'average dendrite length': 4586
              })

In [7]:
insert_result = cells.insert_many([
                 {
                      'name': 'Dentate Gyrus granule cell',
                      'average dendrite length': 3500,
                      'excitatory': True,
                      'channels': ['I Na,t', 'I Na,p', 'I A', 'I K', 'I L high threshold',
                                   'I N', 'I T low threshold', 'I K,Ca', 'I h'],
                      'transmitters': ['Dynorphin', 'Zn2+', 'Glutamate'],
                 },
                 {
                      'name': 'Olfactory granule cell',
                      'average dendrite length': 1000,   # I made this up completely, please don't quote any of these
                      'excitatory': False,
                      'channels': ['I A', 'I N', 'I T low threshold', 'I p,q', 'I CAN', 'I h'],
                      'transmitters': ['NO', 'Gaba'],
                      'region': 'olfactory bulb'
                 }
              ])

In [8]:
print(insert_result.inserted_ids)

[ObjectId('5a66698b51290025ab83f861'), ObjectId('5a66698b51290025ab83f862')]


Note that <tt>insert_one</tt> and <tt>insert_many</tt> return an object which contains the id of the newly created document(s). An <tt>id</tt> may also be specified by including an <tt>_id</tt> key in the dictionary. (Note the underscore.)<br/><br/>We have thus now defined a total of <tt>cells.count()</tt> documents (cells).

In [9]:
print(cells.count())

3


And thus now there really is a collection:

In [10]:
print(mydb.collection_names())

['cells']


<h1>Indexes and uniqueness</h1>

To speed up searches, we can define an index:

In [11]:
cells.create_index('name', unique=True)

'name_1'

The <tt>unique=True</tt> here ensures that there is only one cell in the database with a given name. Attempting to add another e.g. Hippocampus CA1 pyramidal cell will raise a <tt>pymongo.errors.DuplicateKeyError</tt> Exception:

In [12]:
cells.insert_one({
                  'name': 'Hippocampus CA1 pyramidal cell',
                  'average dendrite length': 4586
              })

DuplicateKeyError: E11000 duplicate key error collection: testuser.cells index: name_1 dup key: { : "Hippocampus CA1 pyramidal cell" }

<h1>Querying</h1>

In [13]:
from pprint import pprint
for doc in cells.find({'name': 'Hippocampus CA1 pyramidal cell'}):
    pprint(doc)

{'_id': ObjectId('5a66698a51290025ab83f860'),
 'average dendrite length': 4586,
 'channels': ['I Na,t',
              'I Na,p',
              'I Potassium',
              'I A',
              'I K',
              'I M',
              'I L high threshold',
              'I N',
              'I T low threshold',
              'I p,q',
              'I K,Ca',
              'I h'],
 'name': 'Hippocampus CA1 pyramidal cell',
 'transmitters': ['NO', 'Glutamate']}


If we know there is only one match (or if all we need is one), we can use <tt>find_one</tt> instead and then no iteration is necessary:

In [14]:
pprint(cells.find_one({'transmitters': 'Gaba'}))

{'_id': ObjectId('5a66698b51290025ab83f862'),
 'average dendrite length': 1000,
 'channels': ['I A', 'I N', 'I T low threshold', 'I p,q', 'I CAN', 'I h'],
 'excitatory': False,
 'name': 'Olfactory granule cell',
 'region': 'olfactory bulb',
 'transmitters': ['NO', 'Gaba']}


Note that the search found Gaba as one of multiple transmitters. We can perform more complicated queries using <tt>\$and</tt>, <tt>\$or</tt>, <tt>\$not</tt>, <tt>\$gt</tt>, <tt>\$in</tt>, <tt>\$regex</tt> and other <a href="https://docs.mongodb.com/manual/reference/operator/query/">operators</a>.

In [15]:
for doc in cells.find({'$and': [{'channels': 'I A'}, {'channels': 'I L high threshold'}]}):
    pprint(doc)

{'_id': ObjectId('5a66698a51290025ab83f860'),
 'average dendrite length': 4586,
 'channels': ['I Na,t',
              'I Na,p',
              'I Potassium',
              'I A',
              'I K',
              'I M',
              'I L high threshold',
              'I N',
              'I T low threshold',
              'I p,q',
              'I K,Ca',
              'I h'],
 'name': 'Hippocampus CA1 pyramidal cell',
 'transmitters': ['NO', 'Glutamate']}
{'_id': ObjectId('5a66698b51290025ab83f861'),
 'average dendrite length': 3500,
 'channels': ['I Na,t',
              'I Na,p',
              'I A',
              'I K',
              'I L high threshold',
              'I N',
              'I T low threshold',
              'I K,Ca',
              'I h'],
 'excitatory': True,
 'name': 'Dentate Gyrus granule cell',
 'transmitters': ['Dynorphin', 'Zn2+', 'Glutamate']}


<h3>Example of searching for numeric matches in interval:</h3>

In [16]:
for doc in cells.find({'average dendrite length': {'$lt': 2000}}):
    pprint(doc)

{'_id': ObjectId('5a66698b51290025ab83f862'),
 'average dendrite length': 1000,
 'channels': ['I A', 'I N', 'I T low threshold', 'I p,q', 'I CAN', 'I h'],
 'excitatory': False,
 'name': 'Olfactory granule cell',
 'region': 'olfactory bulb',
 'transmitters': ['NO', 'Gaba']}


<h3>Getting distinct results</h3>

In [17]:
pprint(cells.find().distinct('channels'))

['I A',
 'I K',
 'I K,Ca',
 'I L high threshold',
 'I M',
 'I N',
 'I Na,p',
 'I Na,t',
 'I Potassium',
 'I T low threshold',
 'I h',
 'I p,q',
 'I CAN']


<h3>Sorting results</h3>

We can sort the results of a search based on a field or fields. We use 1 to indicate ascending order, -1 to indicate descending order.

In [18]:
for cell in cells.find().sort([('average dendrite length', 1)]):
    print('{name:35} {average dendrite length:5}'.format(**cell))

Olfactory granule cell               1000
Dentate Gyrus granule cell           3500
Hippocampus CA1 pyramidal cell       4586


In [19]:
for cell in cells.find().sort([('average dendrite length', -1)]):
    print('{name:35} {average dendrite length:5}'.format(**cell))

Hippocampus CA1 pyramidal cell       4586
Dentate Gyrus granule cell           3500
Olfactory granule cell               1000


Part of the reason that this works is that the queries don't return a list; they return a <b>cursor</b> object which allows extracting part of the results.

In [20]:
type(cells.find())

pymongo.cursor.Cursor

This doesn't stop you, however from grabbing a specific item or slice. This is most useful, for example, if in the above we only wanted to receive (from the database) the 2 cells with the longest combined dendritic length. Unlike if a list was sent, using a cursor means no additional data would be sent, so we don't have to deal with bandwidth or sorting it on our local machine. We can also use this to <b>page</b> the results.

In [21]:
for cell in cells.find().sort([('average dendrite length', -1)])[:2]:
    print(cell['name'])

Hippocampus CA1 pyramidal cell
Dentate Gyrus granule cell


<h1>Update</h1>
<h3>Basic use</h3>
<p>Let's declare a region for the cells in the hippocampus, which here we'll select with a regular expression:</p>

In [22]:
cells.update_many({'name': {'$regex': '(Hippocampus|Dentate)'}},
                  {'$set': {'region': 'hippocampus'}})

There is also an <tt>upsert</tt> command that updates entries if it finds a matching one or inserts a new entry if no matching one already exists.

<h3>Numeric updates</h3> 

<p>Suppose for some reason we wanted to convert all dendritic lengths from microns to mm; that is we must multiply them by 0.001. Fortunately, MongoDB provides <a href="https://docs.mongodb.com/manual/reference/operator/update-field/">quantitative</a> update operators such as \$inc and \$mul. </p>

In [23]:
cells.update_many({},
                 {'$mul': {'average dendrite length': 0.001}})

Let's see the results:

In [24]:
for cell in cells.find():
    print('{name:35} {average dendrite length:8} mm'.format(**cell))

Hippocampus CA1 pyramidal cell         4.586 mm
Dentate Gyrus granule cell               3.5 mm
Olfactory granule cell                   1.0 mm


<h3>Updating within an array</h3>
<p>If you use <tt>\$set</tt> a field, you'll replace the entire field. To replace a matched field within an array, use <tt>.\$</tt></p>

In [25]:
cells.update_many({'transmitters': 'Gaba'},
                  {'$set': {'transmitters.$': 'GABA'}}
)

Let's check that (1) Gaba has been replaced by GABA and (2) transmitters is still an array.

In [26]:
for cell in cells.find():
    print('{name:40} {transmitters!r:35}'.format(**cell))

Hippocampus CA1 pyramidal cell           ['NO', 'Glutamate']                
Dentate Gyrus granule cell               ['Dynorphin', 'Zn2+', 'Glutamate'] 
Olfactory granule cell                   ['NO', 'GABA']                     


It worked (see the entry for "Olfactory granule cell" above). Although GABA appears only once in our test dataset, it is a common neurotransmitter.

<h1>Validating documents (applying a schema)</h1>
<p>While one of the advantages of MongoDB is that it does not <i>require</i> specifying a schema,
    it <i>allows</i> specifying one using the (draft) standard <a href="http://json-schema.org/">JSON Schema</a>
    (see link for specification and examples). Letting the database handle validation simplifies your program
    logic.</p>
<p>
    To define a schema for a collection, we must explicitly create the collection (instead of implicitly) and
    specify a <tt>$jsonSchema</tt> validator. Here we define a collection whose documents must contain at least two
    fields and both fields must be strings:
</p>

In [27]:
mydb.create_collection('functions', validator={
    '$jsonSchema': {
        'required': ['name', 'function'],
        'properties': {
            'name': {
                'bsonType': 'string',
                'description': 'name of the brain region; required string'
            },
            'function': {
                'bsonType': 'string',
                'description': 'function of the brain region; required string'
            }
        }        
    }
})

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='testuser'), 'testuser'), 'functions')

Attempting to insert a document without the required properties or with entries of the wrong datatype raises a <tt>WriteError</tt> Exception:

In [28]:
mydb.functions.insert_one({'name': 'olfactory bulb'})

WriteError: Document failed validation

Inserting matching documents, however, succeeds.

In [29]:
mydb.functions.insert_many([
    {
        'name': 'olfactory bulb',
        'function': 'olfaction'
    },
    {
        'name': 'hippocampus',
        'function': 'learning and memory'
    }])

Significantly more complicated validation is possible. See the <a href="http://json-schema.org/">JSON Schema</a> website, the <a href="https://docs.mongodb.com/manual/reference/operator/query/jsonSchema/">MongoDB documentation</a> (in JavaScript not Python, but the syntax is analagous), and elsewhere. You may be asked to build more complicated schema from the information at these links for the homework.

<h1>Joining documents</h1>
<p>Now, let's cross-reference each of our cells with the corresponding brain region functions. (Here we use a <tt>break</tt> to stop printing after the first cell, but otherwise the loop would print out an entry for every cell; try it!)</p>

In [30]:
for cell in cells.aggregate([
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        }
    ]):
    pprint(cell)
    break

{'_id': ObjectId('5a66698a51290025ab83f860'),
 'average dendrite length': 4.586,
 'cellFunctions': [{'_id': ObjectId('5a6669ad51290025ab83f866'),
                    'function': 'learning and memory',
                    'name': 'hippocampus'}],
 'channels': ['I Na,t',
              'I Na,p',
              'I Potassium',
              'I A',
              'I K',
              'I M',
              'I L high threshold',
              'I N',
              'I T low threshold',
              'I p,q',
              'I K,Ca',
              'I h'],
 'name': 'Hippocampus CA1 pyramidal cell',
 'region': 'hippocampus',
 'transmitters': ['NO', 'Glutamate']}


Here the entire function document got embedded into the cell. Sometimes this is what you want. Othertimes you may want to extract just a certain piece of information. To do this use a <b>projection</b>, which will allow you to drop, include, add, or rename fields:

In [31]:
for cell in cells.aggregate([
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        },
        {
            '$project':
            {
                '_id': False,
                'name': True,
                'functions': '$cellFunctions.function'
            }
        }
    ]):
    pprint(cell)


{'functions': ['learning and memory'], 'name': 'Hippocampus CA1 pyramidal cell'}
{'functions': ['learning and memory'], 'name': 'Dentate Gyrus granule cell'}
{'functions': ['olfaction'], 'name': 'Olfactory granule cell'}


<h1>Aggregation</h1>
<p>Joining is a special case of MongoDB's aggregation pipeline which allows you to combine documents, do certain calculations, group data (\$group), etc. For more, see the <a href="http://api.mongodb.com/python/current/examples/aggregation.html">PyMongo aggregation</a> documentation.</p>
<p>Continuing the previous example, suppose I only wanted to know the functions for cells in the hippocampus. In that case, I'll begin by doing a <tt>\$match</tt> to find just those cells before doing the <tt>\$lookup</tt>. The arguments to <tt>\$match</tt> are the same as you would send to <tt>find_many</tt>.</p>

In [32]:
for cell in cells.aggregate([
        {
            '$match':
            {
                'region': 'hippocampus'
            }
        },
        {
            '$lookup':
            {
                'from': 'functions',
                'localField': 'region',
                'foreignField': 'name',
                'as': 'cellFunctions'
            }
        },
        {
            '$project':
            {
                '_id': False,
                'name': True,
                'functions': '$cellFunctions.function'
            }
        }
    ]):
    pprint(cell)


{'functions': ['learning and memory'], 'name': 'Hippocampus CA1 pyramidal cell'}
{'functions': ['learning and memory'], 'name': 'Dentate Gyrus granule cell'}


Suppose we want to get a count of the number of times a given channel appears. We begin by <tt>\$unwind</tt>ing the channels array. This creates one virtual document for each channel. (If we didn't do this, we'd count the number of times a particular set of channels appears. This is interesting information, but not what we want here.)

In [33]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        }
    ]))

[{'_id': ObjectId('5a66698a51290025ab83f860'),
  'average dendrite length': 4.586,
  'channels': 'I Na,t',
  'name': 'Hippocampus CA1 pyramidal cell',
  'region': 'hippocampus',
  'transmitters': ['NO', 'Glutamate']},
 {'_id': ObjectId('5a66698a51290025ab83f860'),
  'average dendrite length': 4.586,
  'channels': 'I Na,p',
  'name': 'Hippocampus CA1 pyramidal cell',
  'region': 'hippocampus',
  'transmitters': ['NO', 'Glutamate']},
 {'_id': ObjectId('5a66698a51290025ab83f860'),
  'average dendrite length': 4.586,
  'channels': 'I Potassium',
  'name': 'Hippocampus CA1 pyramidal cell',
  'region': 'hippocampus',
  'transmitters': ['NO', 'Glutamate']},
 {'_id': ObjectId('5a66698a51290025ab83f860'),
  'average dendrite length': 4.586,
  'channels': 'I A',
  'name': 'Hippocampus CA1 pyramidal cell',
  'region': 'hippocampus',
  'transmitters': ['NO', 'Glutamate']},
 {'_id': ObjectId('5a66698a51290025ab83f860'),
  'average dendrite length': 4.586,
  'channels': 'I K',
  'name': 'Hippocampus

Now let's <tt>\$group</tt> by channels (which is now a single item) and create a new count field that sums up 1 for each document:

In [34]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        },
        {
            '$group': {
                '_id': '$channels',
                'count': {'$sum': 1}
            }
        }
    ]))

[{'_id': 'I CAN', 'count': 1},
 {'_id': 'I h', 'count': 3},
 {'_id': 'I K,Ca', 'count': 2},
 {'_id': 'I Na,p', 'count': 2},
 {'_id': 'I A', 'count': 3},
 {'_id': 'I N', 'count': 3},
 {'_id': 'I Na,t', 'count': 2},
 {'_id': 'I T low threshold', 'count': 3},
 {'_id': 'I K', 'count': 2},
 {'_id': 'I L high threshold', 'count': 2},
 {'_id': 'I Potassium', 'count': 1},
 {'_id': 'I p,q', 'count': 2},
 {'_id': 'I M', 'count': 1}]

We can <tt>\$sort</tt> the channels by count. A basic principle is that <b>it's better to let your database do as much of the data processing as possible than to transfer data to Python and process it there</b>.

In [35]:
list(cells.aggregate([
        {
            '$unwind': '$channels'
        },
        {
            '$group': {
                '_id': '$channels',
                'count': {'$sum': 1}
            }
        },
        {
            '$sort': {'count': -1}
        }
    ]))

[{'_id': 'I h', 'count': 3},
 {'_id': 'I A', 'count': 3},
 {'_id': 'I N', 'count': 3},
 {'_id': 'I T low threshold', 'count': 3},
 {'_id': 'I K,Ca', 'count': 2},
 {'_id': 'I Na,p', 'count': 2},
 {'_id': 'I Na,t', 'count': 2},
 {'_id': 'I K', 'count': 2},
 {'_id': 'I L high threshold', 'count': 2},
 {'_id': 'I p,q', 'count': 2},
 {'_id': 'I CAN', 'count': 1},
 {'_id': 'I Potassium', 'count': 1},
 {'_id': 'I M', 'count': 1}]

Just like with the <tt>sort</tt> method we saw before: the -1 in the <tt>\$sort</tt> command above means to sort in descending order. To sort in ascending order, replace the -1 with a 1.

Warning: MongoDB has a limit on the amount of memory it will use for a sort. If your query requires too much, a straightforward sort will fail, although there are work-arounds.

An even more flexible solution for data processing is to write your own <tt>map</tt> and <tt>reduce</tt> functions in JavaScript (yes, even though you're otherwise working in Python) and pass them to map_reduce. A simple example is <a href="http://api.mongodb.com/python/current/examples/aggregation.html#map-reduce">here</a>.

<h1>Deleting documents, etc</h1>

In [36]:
result = cells.delete_many({'name': {'$regex': 'Dentate'}})

In [37]:
result.deleted_count

1

There is also a <tt>delete_one</tt> method that deletes only one match.<br/><br/>To delete an entire collection, use, e.g.:

In [38]:
mydb.drop_collection('functions')

{'nIndexesWas': 1, 'ns': 'testuser.functions', 'ok': 1.0}

In [39]:
mydb.drop_collection('cells')

{'nIndexesWas': 2, 'ns': 'testuser.cells', 'ok': 1.0}